<img src="images/quantum_algorithms_tutorials.png" alt="drawing" width="100" align="left"/>

<h1 align="center">
	Variational quantum eigensolver
</h1>

Variational quantum eigensolver (VQE) is a hybrid quantum-classical algorithm [[1](https://www.nature.com/articles/ncomms5213?origin=ppub)] that finds the smallest eigenvalue (and corresponding eigenvector) of a given Hamiltonian. One of the main applications of the algorithm is finding ground state energy of molecules. It has a big advantage over [IPEA](https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/iterative_phase_estimation.ipynb) (iterative phase estimation algorithm) and PEA (phase estimation algorithm), that also can be used for finding the ground state energy of a molecule. The main advantage is that VQE uses  much smaller circuit depths (or gates) then IPEA and PEA, what is very important for NISQ (Noisy Intermediate-Scale Quantum) era quantum computation. In the NISQ era (now!) we are working with qubits that are very noisy because they are not isolated from the environment well enough. Thus, there is small and finite time to work with qubits until they will be "spoiled", because of the environment, imperfect gates and etc. This restriction gives a big advantage to those algorithms (like VQE) that are using small depth circuits. 

The idea of the VQE algorithm is as follows. We have a Hamiltonian that can be expressed by the sum of tensor products of Pauli operators (Pauli terms):

$$H = 0.4 \cdot IX + 0.6 \cdot IZ + 0.8 \cdot XY.$$

For a given $\left| \psi \right\rangle$ <font size="0.1">$\left| \psi \right\rangle$</font> we want to measure the expectation value of the Hamiltonian:

$$\left\langle H \right\rangle = \left\langle \psi \right| H \left| \psi \right\rangle = 0.4 \cdot \left\langle \psi \right| IX \left| \psi \right\rangle + 0.6 \cdot \left\langle \psi \right| IZ \left| \psi \right\rangle + 0.8 \cdot \left\langle \psi \right| XY \left| \psi \right\rangle.$$

How one can see the $\left\langle H \right\rangle$ expectation value could be computed by adding the expectation values of its parts (Pauli terms). The algorithm does exactly that. It constructs a quantum circuit for each Pauli term and computes the expectation value of the corresponding Pauli term. Then, the algorithm sums all calculated expectation values of Pauli terms and obtains the expectation value of $H$. In this algorithm, we will do this routine of estimating the expectation value of $H$ over and over again for different trial wavefunctions (ansatz states) $\left| \psi \right\rangle$. 

It is known that the eigenvector $\left| \psi_g \right\rangle$ that minimizes the expectation value $\left\langle H \right\rangle$ corresponds to the eigenvector of $H$ that has the smallest eigenvalue [[1](https://www.nature.com/articles/ncomms5213?origin=ppub)]. So, basically we can try all possible trial wavefunctions $\left| \psi \right\rangle$s to find the $\left| \psi_g \right\rangle$ that has the smallest expectation value. Here the question is how we create those trial states? In the algorithm, the trial states are created from a parametrized circuit. By changing the parameters one obtains different wavefunctions (ansatz states) [[1](https://www.nature.com/articles/ncomms5213?origin=ppub)]. If your circuit with its parameters is good enough you will have access to more or less all possible ansatz states [[2](https://www.mustythoughts.com/post/variational-quantum-eigensolver-explained)]. Otherwise, if the circuit will not have a possibility to generate our desired $\left| \psi_g \right\rangle$ it will be impossible to find the right solution.

The parameters of the state preparation circuit are controlled by a classical computer. At each step, the classical computer will change the parameters by using some optimization method in order to create an ansatz state that will have a smaller expectation value then previous ansatz states had. This way the classical computer and the quantum computer are working together to archive the goal of the algorithm (to find the ground state energy). That's way, VQE is a quantum-classical hybrid algorithm.

<img src="images/vqe_parts.png" alt="drawing" width="600"/>

This image is taken from the [[1](https://www.nature.com/articles/ncomms5213?origin=ppub)] paper. Here are the main parts of the VQE algorithm. The *Classical feedback decision* is some optimization method that changes the parameters of the *Quantum state preparation*. With different *Quantum modules* the algorithm calculates expectation values of each Pauli term and then it sums them by using *Classical adder* via classical computer. Then the algorithm returns to *Classical feedback decision* to choose better parameters for the *Quantum state preparation*. VQE repeats this procedure until the optimization method is satisfied with the obtained result. Note, that we will use these names in the image as method names in our code.

Now let's see how the algorithm works. Firstly, we should import all the packages that we will use.

In [1]:
import numpy as np
from random import random
from scipy import array
from scipy.optimize import minimize

from qiskit import *
from qiskit.extensions.standard import *
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import ExactEigensolver

For simplicity we will consider one qubit Hamiltonian consisting of a sum of 4 Pauli operators:

$$H = a \cdot I + b \cdot Z + c \cdot X + d \cdot Y,$$

here are matrix representations of each Pauli operators:

$$
I = \begin{pmatrix}
1 & 0\\
0 & 1
\end{pmatrix},
\qquad
Z = \begin{pmatrix}
1 & 0\\
0 & -1
\end{pmatrix},
\qquad
X = \begin{pmatrix}
0 & 1\\
1 & 0
\end{pmatrix},
\qquad
Y = \begin{pmatrix}
0 & -i\\
i & 1
\end{pmatrix}.
$$

The following method creates $H$ for a given $a$, $b$, $c$, $d$ coefficients.

In [2]:
def hamiltonian_operator(a, b, c, d):
    """
    Creates a*I + b*Z + c*X + d*Y pauli sum 
    that will be our Hamiltonian operator.
    
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "I"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "Z"},
                   {"coeff": {"imag": 0.0, "real": c}, "label": "X"},
                   {"coeff": {"imag": 0.0, "real": d}, "label": "Y"}
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

We define $a$, $b$, $c$, $d$ coefficients as random numbers. Thus our Hamiltonian will be a random generated Hamiltonian.

In [3]:
a, b, c, d = (random(), random(), random(), random())
H = hamiltonian_operator(a, b, c, d)

Here we are using qiskit's ```ExactEigensolver``` class that will find the smallest eigenvalue of the given Hamiltonian via a classical algorithm. This value will be used as a reference for comparing it with VQE result.

In [4]:
exact_eigensolver = ExactEigensolver(H, k=2)
result_ee = exact_eigensolver.run()
reference_energy = result_ee['energy']
print('The exact ground state energy is: {}'.format(reference_energy))

The exact ground state energy is: -0.8632690166995267


Now we need to create "good" ansatz states. First of all, note that every possible wavefunction $\left| \psi \right\rangle$ can be presented as a vector: 
$$
\left| \psi \right\rangle = \begin{pmatrix}
\cos{\left( \theta/2 \right)}\\
e^{i \varphi} \cdot \sin{\left( \theta/2 \right)}
\end{pmatrix},
$$

where the numbers $\theta$ and $\varphi$ define a point on the unit three-dimensional sphere that is called Bloch sphere and is presented in the following picture from this book [[3](https://www.cambridge.org/am/academic/subjects/physics/quantum-physics-quantum-information-and-quantum-computation/quantum-computation-and-quantum-information-10th-anniversary-edition?format=HB)].

<img src="images/bloch_sphere.png" alt="drawing" width="300"/>

For one qubit Hamiltonian, a "good" quantum state preparation circuit should be able to generate all possible states in the Bloch sphere [[2](https://www.mustythoughts.com/post/variational-quantum-eigensolver-explained)]. Before quantum state preparation, our qubit is in $\left| 0 \right\rangle = \begin{pmatrix}
1\\
0
\end{pmatrix}$
state. This corresponds to the vertical position of the vector in the Bloch sphere. In order to generate any possible $\left| \psi \right\rangle$ we will apply $R_x(t_1)$ and $R_y(t_2)$ gates on the $\left| 0 \right\rangle$ initial state: $R_y(t_2) R_x(t_1) \left| 0 \right\rangle = \left| \psi \right\rangle$. $R_x(t_1)$ corresponds to the rotation in the Bloch sphere around the *x* axis and $R_y(t_2)$ the rotation around the *y* axis. With these two rotations, one can have access to any point in the Bloch sphere. Here we show the matrix forms of $R_x(t_1)$ and $R_y(t_2)$ gates:

$$
R_x(t_1) = \begin{pmatrix}
cos(\frac{t_1}{2}) & -i \cdot sin(\frac{t_1}{2})\\
-i \cdot sin(\frac{t_1}{2}) & cos(\frac{t_1}{2})
\end{pmatrix},
\qquad
R_y(t_2) = \begin{pmatrix}
cos(\frac{t_2}{2}) & -sin(\frac{t_2}{2})\\
sin(\frac{t_2}{2}) & cos(\frac{t_2}{2})
\end{pmatrix}.
$$

These two gates with there parameters ($t_1$ and $t_2$) will generate for us the trial (ansatz) wavefunctions. The two parameters will be in control of the Classical Computer and its optimization model.

The following method corresponds to the quantum state preparation circuit for given parameters ($t_1$ and $t_2$).

In [5]:
def quantum_state_preparation(circuit, parameters):
    q = circuit.qregs[0] # q is the quantum register where the info about qubits is stored
    circuit.rx(parameters[0], q[0]) # q[0] is our one and only qubit XD
    circuit.ry(parameters[1], q[0])
    return circuit

Now we come to the part of the algorithm where we should do individual/separate measurements of expectation values of the Pauli terms. For one qubit case, there are only four possible Pauli terms: 4 basic Pauli operators $I$, $X$, $Y$, $Z$.

$$\left\langle H \right\rangle = \left\langle \psi \right| H \left| \psi \right\rangle = a \cdot \left\langle \psi \right| I \left| \psi \right\rangle + b \cdot \left\langle \psi \right| Z \left| \psi \right\rangle + c \cdot \left\langle \psi \right| X \left| \psi \right\rangle + d \cdot \left\langle \psi \right| Y \left| \psi \right\rangle.$$

For $I$ operator the  expectation value is always unity: $\left\langle \psi \right| I \left| \psi \right\rangle = \left\langle \psi \right| \left| \psi \right\rangle = 1$. So, its contribution to the overall expectation value $\left\langle H \right\rangle$ will be equal to $a \cdot \left\langle \psi \right| I \left| \psi \right\rangle = a$.

For rest of the Pauli operators, we should make the following remark: every quantum state $\left| \psi \right\rangle$ can be represented via different sets of basis vectors:

$$\left| \psi \right\rangle = c_1^z \cdot \left| 0 \right\rangle + c_2^z \cdot \left| 1 \right\rangle = c_1^x \cdot \left| + \right\rangle + c_2^x \cdot \left| - \right\rangle = c_1^y \cdot \left| +i \right\rangle + c_2^y \cdot \left| -i \right\rangle,$$

where

\begin{align*}
&\text{Z eigenvectors} \qquad
\left| 0 \right\rangle = \begin{pmatrix}
1\\
0
\end{pmatrix},
&&\left| 1 \right\rangle = \begin{pmatrix}
0\\
1
\end{pmatrix},
\\
&\text{X eigenvectors} \qquad
\left| + \right\rangle = \frac{1}{\sqrt{2}} \begin{pmatrix}
1\\
1
\end{pmatrix},
&&\left| - \right\rangle = \frac{1}{\sqrt{2}} \begin{pmatrix}
1\\
-1
\end{pmatrix},
\\
&\text{Y eigenvectors} \qquad
\left| +i \right\rangle = \frac{1}{\sqrt{2}} \begin{pmatrix}
1\\
i
\end{pmatrix}, 
&&\left| -i \right\rangle = \frac{1}{\sqrt{2}} \begin{pmatrix}
1\\
-i
\end{pmatrix}.
\end{align*}

The first presented eigenvectors for each Pauli has an eigenvalue equal to $+1$: $Z \left| 0 \right\rangle = +1\left| 0 \right\rangle$, $X \left| + \right\rangle = +1\left| + \right\rangle$, $Y \left| +i \right\rangle = +1\left| +i \right\rangle$. And the second presented eigenvectors for each Pauli has an eigenvalue equal to $-1$: $Z \left| 1 \right\rangle = -1\left| 1 \right\rangle$, $X \left| - \right\rangle = -1\left| - \right\rangle$, $Y \left| -i \right\rangle = -1\left| -i \right\rangle$. Now, let's calculate the expectation values of these Pauli operators: 

\begin{align*}
\left\langle \psi \right| Z \left| \psi \right\rangle &= \left( {c_1^z}^* \cdot \left\langle 0 \right| + {c_2^z}^* \cdot \left\langle 1 \right| \right) Z \left( c_1^z \cdot \left| 0 \right\rangle + c_2^z \cdot \left| 1 \right\rangle \right) = {\left| c_1^z \right|}^2 - {\left| c_2^z \right|}^2,
\\
\left\langle \psi \right| X \left| \psi \right\rangle &= \left( {c_1^x}^* \cdot \left\langle + \right| + {c_2^x}^* \cdot \left\langle - \right| \right) X \left( c_1^x \cdot \left| + \right\rangle + c_2^x \cdot \left| - \right\rangle \right) = {\left| c_1^x \right|}^2 - {\left| c_2^x \right|}^2,
\\
\left\langle \psi \right| Y \left| \psi \right\rangle &= \left( {c_1^y}^* \cdot \left\langle +i \right| + {c_2^y}^* \cdot \left\langle -i \right| \right) Y \left( c_1^y \cdot \left| +i \right\rangle + c_2^y \cdot \left| -i \right\rangle \right) = {\left| c_1^y \right|}^2 - {\left| c_2^y \right|}^2,
\end{align*}

where we take into account that the inner product of orthonormal vectors is 0 (e.g. $\left\langle 0 \right| \left| 1 \right\rangle = 0$, $\left\langle + \right| \left| - \right\rangle = 0$, $\left\langle +i \right| \left| -i \right\rangle = 0$). But what are these $\left| c \right|^2$s? The ${\left| c_1^z \right|}^2$ and ${\left| c_2^z \right|}^2$ are by definition the probabilities that after Z basis measurement (measuring is it $\left| 0 \right\rangle$ or is it $\left| 1 \right\rangle$) the quantum state $\left| \psi \right\rangle$ will become $\left| 0 \right\rangle$ or $\left| 1 \right\rangle$ respectively. In order to find that value, we should run our program with our trial $\left| \psi \right\rangle$  wavefunction and do $Z$ measurement on the qubit $N$ times (its named ```shots``` in the code). The probability of finding the qubit after measurment in $\left| 0 \right\rangle$ state will be equal to ${\left| c_1^z \right|}^2 = \frac{n_0}{N}$, where $n_0$ is the number of the $\left| 0 \right\rangle$ state measurments. Similarly, ${\left| c_2^z \right|}^2 = \frac{n_1}{N}$, where $n_1$ is the number of the $\left| 1 \right\rangle$ state measurments. Thus, the final expectation value will be $\left\langle Z \right\rangle = \frac{n_0 - n_1}{N}$.

For $\left\langle X \right\rangle = \frac{n_+ - n_-}{N}$ and $\left\langle Y \right\rangle = \frac{n_{+i} - n_{-i}}{N}$ the expectation value estimation procedure stays the same. Here $n_+$ and $n_-$ are numbers of measurements in X basis that corresponds to $\left| + \right\rangle$ or $\left| - \right\rangle$ outcomes respectively. And $n_{+i}$ and $n_{-i}$ are numbers of measurements in $Y$ basis that corresponds to $\left| +i \right\rangle$ or $\left| -i \right\rangle$ outcomes respectively. The difficulty comes from the fact that one may have the possibility to measure only in the $Z$ basis. To solve this difficulty we still do a $Z$ basis measurement, but, before that, we apply specific operators to the $\left| \psi \right\rangle$ state. We try to apply such an operator that after measuring the probability of $\left| 0 \right\rangle$ outcome will be equal to the probability of $\left| + \right\rangle$ $\left( \left| +i \right\rangle \right)$ outcome. And the probability of $\left| 1 \right\rangle$ outcome will be equal to the probability of $\left| - \right\rangle$ $\left( \left| -i \right\rangle \right)$ outcome. Let's define such operators:

$$H_{gate} \left| \psi \right\rangle = H_{gate} \left( c_1^x \cdot \left| + \right\rangle + c_2^x \cdot \left| - \right\rangle \right) = c_1^x \cdot \left| 0 \right\rangle + c_2^x \cdot \left| 1 \right\rangle,$$

where $H_{gate}$ is an operator such that $H_{gate} \left| + \right\rangle = \left| 0 \right\rangle$ and $H_{gate} \left| - \right\rangle = \left| 1 \right\rangle$.

$$Y_{gate} \left| \psi \right\rangle = Y_{gate} \left( c_1^y \cdot \left| +i \right\rangle + c_2^y \cdot \left| -i \right\rangle \right) = c_1^y \cdot \left| 0 \right\rangle + c_2^y \cdot \left| 1 \right\rangle,$$

where $Y_{gate}$ is an operator such that $Y_{gate} \left| +i \right\rangle = \left| 0 \right\rangle$ and $Y_{gate} \left| -i \right\rangle = \left| 1 \right\rangle$.

This kind of operators can be easily found:

$$
H_{gate} = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1\\
1 & -1
\end{pmatrix},
\qquad
Y_{gate} = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & -i\\
1 & i
\end{pmatrix}.
$$

BTW the $H_{gate}$ is the well known Hadamard gate (its common notation is $H$, but we don't use this notation because we already have used the $H$ for the Hamiltonian operator). In the following code you can see an implementation of these gates via qiskit's ```u2``` gate:

$$
u2(\varphi, \lambda) = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & -e^{i \lambda}\\
e^{i \varphi} & e^{i (\varphi + \lambda)}
\end{pmatrix}.
$$

In [6]:
H_gate = U2Gate(0, np.pi).to_matrix()
print("H_gate:")
print((H_gate * np.sqrt(2)).round(5))

Y_gate = U2Gate(0, np.pi/2).to_matrix()
print("Y_gate:")
print((Y_gate * np.sqrt(2)).round(5))

H_gate:
[[ 1.+0.j  1.-0.j]
 [ 1.+0.j -1.+0.j]]
Y_gate:
[[ 1.+0.j -0.-1.j]
 [ 1.+0.j  0.+1.j]]


Finally, we are ready to go! In this code, we define a method that will create a circuit with its specific measurement ($Z$ or $X$ or $Y$ measurments).

In [7]:
def vqe_circuit(parameters, measure):
    """
    Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'Z' stands for Z measurement.
    :return: quantum circuit.
    """
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    circuit = QuantumCircuit(q, c)

    # quantum state preparation
    circuit = quantum_state_preparation(circuit, parameters)

    # measurement
    if measure == 'Z':
        circuit.measure(q[0], c[0])
    elif measure == 'X':
        circuit.u2(0, np.pi, q[0])
        circuit.measure(q[0], c[0])
    elif measure == 'Y':
        circuit.u2(0, np.pi/2, q[0])
        circuit.measure(q[0], c[0])
    else:
        raise ValueError('Not valid input for measurement: input should be "X" or "Y" or "Z"')

    return circuit

We are going to work with python dictionaries and the following method will help us to avoid some errors. It will return the value (number of $\left| 0 \right\rangle$ state measurements) of a corresponding key (e.g ```'0'```), if it exists, else return $0$ (never measured ```'0'``` state).

In [8]:
def get_or_else_zero(d: dict, key: str):
    """
    Utility for working with dictionaries. If key is missing
    than return 0 otherwise the corresponding value.
    :param dict: the dictionary.
    :param key: key (string) in interest.
    :return: 0 or value of corresponding key.
    """
    value = 0
    if key in d:
        value = d[key]
    return value

Here is the quantum module part. The ```quantum_module``` finds the expectation values of a Pauli operator.

In [9]:
def quantum_module(parameters, measure):
    # measure
    if measure == 'I':
        return 1
    elif measure == 'Z':
        circuit = vqe_circuit(parameters, 'Z')
    elif measure == 'X':
        circuit = vqe_circuit(parameters, 'X')
    elif measure == 'Y':
        circuit = vqe_circuit(parameters, 'Y')
    else:
        raise ValueError('Not valid input for measurement: input should be "I" or "X" or "Z" or "Y"')
    
    shots = 1000
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    expectation_value = (get_or_else_zero(counts, '0') - get_or_else_zero(counts,'1')) / shots
    
    return expectation_value

The ```pauli_operator_to_dict``` creates a  dictionary from the ```WeightedPauliOperator``` object (the Hamiltonian is encoded in this object). 

In [10]:
def pauli_operator_to_dict(pauli_operator):
    """
    from WeightedPauliOperator return a dict:
    {I: 0.7, X: 0.6, Z: 0.1, Y: 0.5}.
    :param palui_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}

    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff

    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

The following is the main method that takes parameters for ansatz state preparation and returns the corresponding expectation value of the Hamiltonian. For each Pauli term, we create separate quantum modules that calculate the expectation value of each Pauli. Then, all expectation values of Pauli operators multiplied by there corresponding coefficients ($a$, $b$, $c$, $d$) are summed.

In [11]:
def vqe(parameters):
        
    # quantum_modules
    quantum_module_I = get_or_else_zero(pauli_dict, 'I') * quantum_module(parameters, 'I')
    quantum_module_Z = get_or_else_zero(pauli_dict, 'Z') * quantum_module(parameters, 'Z')
    quantum_module_X = get_or_else_zero(pauli_dict, 'X') * quantum_module(parameters, 'X')
    quantum_module_Y = get_or_else_zero(pauli_dict, 'Y') * quantum_module(parameters, 'Y')
    
    # summing the measurement results
    classical_adder = quantum_module_I + quantum_module_Z + quantum_module_X + quantum_module_Y
    
    return classical_adder

This is the final part of the code where we are minimizing (optimizing) the returned value from the ```vqe``` method by changing parameters for the quantum state preparation circuit (trial/ansatz $\left| \psi \right\rangle$).

In [12]:
parameters_array = array([np.pi, np.pi])
tol = 1e-3 # tolerance for optimization precision.

vqe_result = minimize(vqe, parameters_array, method="Powell", tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))

The exact ground state energy is: -0.8632690166995267
The estimated ground state energy from VQE algorithm is: -0.8640435130915292


This is it! VQE algorithm for one qubit. But what should be done if there is a Hamiltonian for multiple qubits? For example, the Hamiltonian for the two-qubit system could look like this:

$$\left\langle H \right\rangle = \left\langle \psi \right| H \left| \psi \right\rangle = 0.4 \cdot \left\langle \psi \right| IX \left| \psi \right\rangle + 0.6 \cdot \left\langle \psi \right| IZ \left| \psi \right\rangle + 0.8 \cdot \left\langle \psi \left| XY \right| \psi \right\rangle.$$

This time the algorithm should calculate the expectation value of each Pauli product. The most not trivial case is the $XY$ Pauli term and we will consider only this case. Note that for tensor products of operators like $XY$ Pauli term (actually $XY = X \otimes Y$):

$$U_1 \otimes U_2 \left| \psi_1 \right\rangle \otimes \left| \psi_2 \right\rangle =  U_1 \left| \psi_1 \right\rangle \otimes U_2 \left| \psi_2 \right\rangle,$$

were $U_1$ and $U_2$ are some unitary operators acting on two qubits with separate $\left| \psi_1 \right\rangle$ and $\left| \psi_2 \right\rangle$ wavefunctions. This way it can be shown that tensor products of the eigenvectors of $X$ and $Y$ are the eigenvectors of $XY$ Pauli term. It is true for any Pauli product term. Also, notice that all eigenvectors of  Pauli terms have eigenvalues equal to either $+1$ or $-1$. The eigenvectors of $XY$ that have $+1$ eigenvalue are $\left| + \right\rangle \otimes \left| +i \right\rangle = \left| ++i \right\rangle$, $\left| - \right\rangle \otimes \left| -i \right\rangle = \left| --i \right\rangle$, and the eigenvectors that have $-1$ eigenvalue are $\left| + \right\rangle \otimes \left|-+i \right\rangle = \left| +-i \right\rangle$, $\left| + \right\rangle \otimes \left| -i \right\rangle = \left| +-i \right\rangle$. As an example:

$$XY \left| + -i \right\rangle= X \otimes Y \left| + \right\rangle \otimes \left| -i \right\rangle =  X \left| + \right\rangle \otimes Y \left| -i \right\rangle = 
-\left| + \right\rangle \otimes \left| -i \right\rangle = - \left| + -i \right\rangle.$$

The measurement logic stays the same. So, for $\left\langle XY \right\rangle$ one should apply $H_{gate}$ on the first qubit and $Y_{gate}$ on the second qubit (in tensor product notation $H_{gate} \otimes Y_{gate}$) before $Z$ basis measurement. To see that we should represent the combined quantum state $\left| \psi \right\rangle = \left| \psi_1 \right\rangle \otimes \left| \psi_2 \right\rangle$ in the basis of the eigenvectors of $XY$ operator:

$$H_{gate} \otimes Y_{gate} \left| \psi \right\rangle = H_{gate} \otimes Y_{gate} \left( c_1^{XY}\left| ++i \right\rangle + c_2^{XY}\left| +-i \right\rangle + c_3^{XY}\left| -+i \right\rangle + c_4^{XY}\left| --i \right\rangle \right) \\
= c_1^{XY}\left| 00 \right\rangle + c_2^{XY}\left| 01 \right\rangle + c_3^{XY}\left| 10 \right\rangle + c_4^{XY}\left| 11 \right\rangle,$$

where $\left| c \right|^2$ are probabilities of obtaining the corresponding eigenvectors of $XY$ after the measurement. By doing similar calculations as we did for single Paulis, it can be shown that the expectation value $\left\langle \psi \right| XY \left| \psi \right\rangle$ is equal to the sum of $\left| c \right|^2$s with $-1$ coefficient for those eigenvectors which eigenvalue is $-1$ and with $+1$ coefficient for those eigenvectors which eigenvalue is $-1$.

This way one can scale this solution for bigger Hamiltonians.

[1] [A. Peruzzo et al., Nature Communications, "A variational eigenvalue solver on a photonic quantum processor" (2014).](https://www.nature.com/articles/ncomms5213?origin=ppub)

[2] [Michał Stęchły, "Variational Quantum Eigensolver explained"](https://www.mustythoughts.com/post/variational-quantum-eigensolver-explained).

[3] [M.A. Nielsen, I.L. Chuang, Cambridge University Press New York, "Quantum Computation and Quantum Information: 10th Anniversary Edition
10th" (2011)](https://www.cambridge.org/am/academic/subjects/physics/quantum-physics-quantum-information-and-quantum-computation/quantum-computation-and-quantum-information-10th-anniversary-edition?format=HB)